In [96]:
#########################################
# Test functions and non-permanant code #
# Author: Jarrett Reimers               #
# Keep as a reference, do not edit      #
#########################################
import pandas as pd
import os
import datetime
from query import Query

In [98]:
data = pd.read_csv("data/2015/5/1.csv")

starttimes = []
for starttime in data['starttime']:
  starttimes.append(datetime.datetime.strptime(starttime, "%m/%d/%Y %H:%M:%S"))
data['starttime'] = starttimes

data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,415,2015-05-01 00:00:11,5/1/2015 00:07:06,477,W 41 St & 8 Ave,40.756405,-73.990026,442,W 27 St & 7 Ave,40.746647,-73.993915,17012,Subscriber,1981.0,1
1,1523,2015-05-01 00:00:28,5/1/2015 00:25:52,293,Lafayette St & E 8 St,40.730287,-73.990765,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,17390,Customer,NaN,0
2,642,2015-05-01 00:00:52,5/1/2015 00:11:35,380,W 4 St & 7 Ave S,40.734011,-74.002939,507,E 25 St & 2 Ave,40.739126,-73.979738,15003,Subscriber,1990.0,1
3,367,2015-05-01 00:00:48,5/1/2015 00:06:55,537,Lexington Ave & E 24 St,40.740259,-73.984092,280,E 10 St & 5 Ave,40.733320,-73.995101,14788,Subscriber,1978.0,1
4,2734,2015-05-01 00:00:51,5/1/2015 00:46:26,426,West St & Chambers St,40.717548,-74.013221,327,Vesey Pl & River Terrace,40.715338,-74.016584,21068,Subscriber,1956.0,2


In [100]:
query = Query()
start_time = query.get_datetime(2015, 5, 13, 0, 0, 0)
end_time = query.get_datetime(2015, 5, 16, 0, 0, 0)
three_day = query.select_time(data, start_time, end_time)
three_day.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
364407,298,2015-05-13 00:00:04,5/13/2015 00:05:03,357,E 11 St & Broadway,40.732618,-73.991580,461,E 20 St & 2 Ave,40.735877,-73.982050,21195,Subscriber,1990.0,1
364408,482,2015-05-13 00:00:13,5/13/2015 00:08:15,493,W 45 St & 6 Ave,40.756800,-73.982912,519,Pershing Square North,40.751873,-73.977706,17503,Customer,NaN,0
364409,132,2015-05-13 00:00:26,5/13/2015 00:02:39,432,E 7 St & Avenue A,40.726218,-73.983799,236,St Marks Pl & 2 Ave,40.728419,-73.987140,15975,Subscriber,1993.0,1
364410,119,2015-05-13 00:00:22,5/13/2015 00:02:21,515,W 43 St & 10 Ave,40.760094,-73.994618,495,W 47 St & 10 Ave,40.762699,-73.993012,18659,Subscriber,1987.0,1
364411,1017,2015-05-13 00:00:29,5/13/2015 00:17:26,488,W 39 St & 9 Ave,40.756458,-73.993722,426,West St & Chambers St,40.717548,-74.013221,19100,Subscriber,1979.0,1
